<img src="figures/logo_large.jpg" alt="SVDS" width="590" align="left">

# Proof of concept: detection of a train’s passing and direction from video

## Introduction

I have hidden the actual executable code for simplicity, as it is not required to understand the development of the algorithm, but feel free to press the `toggle code` button at the top of the page if you would like to follow along. Additionally, the appendix contains code for running the motion detection portion of the algorithm on a video feed from your own computer camera so that you can get a feel for how each step works through experimentation. 

We developed this algorithm in Python, using the `pandas` and `numpy` packages. All image/video processing steps are performed using [OpenCV](http://opencv.org/). OpenCV is one of the most used and effective packages for image processing and computer vision. It was first launched in 1999 and the third version was just released at the end of 2015. It is written in C++, but it has bindings in Python, Java, and MATLAB/OCTAVE. In our work, we use the Python bindings of OpenCV3. 

*Tip*: If installing for Python, I would *highly* recommend doing so through [Anaconda](http://docs.continuum.io/anaconda/index), as installing it through `pip` or other methods can be *very* painful. The command for installation using Anaconda is:

`conda install -c https://conda.binstar.org/menpo opencv3`

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2 #OpenCV

## The algorithm

As mentioned, we developed this proof of concept (POC) using clips of videos, hand selected, of a Caltrain passing our office. To demonstrate the steps of the algorithm, I will use the following clip:

In [4]:
"""Here we load the video clip from an mp4 into a list of 
images to be used for the demonstration of the algorithm."""

# Load the video as a series of frames using the helper function in trainspotting.py
video_file = cv2.VideoCapture('video/orig.mp4')

# Read iterates through the frames in the video object and returns:
# 1. Logical - True if a frame has been read
# 2. Image - an array with the current frame
read_file, frame = video_file.read()
original = []
while read_file:
    # Going to grab the frame and create a list for future use 
    original.append(frame)
    
    # Use imshow to play video
    cv2.imshow('Original video',frame)
    
    # Get next frame 
    read_file, frame = video_file.read()
    
    # Pause and allow for "q" button to stop video
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Original video')

In [10]:
"""Sometimes, OpenCV has problems with ingesting 
.avi video files. If it does, you can use this code
to load the file."""
import re
a = []
b = []
original = []
with open('video/test.avi') as f:
    bytes = f.read()
p = re.compile(r'\xff\xd8')
p2 = re.compile(r'\xff\xd9')
for m in p.finditer(bytes):
    a += [m.start()]
for m in p2.finditer(bytes):
    b += [m.start()]
for c, d in zip(a, b):
    jpg = bytes[c:d+2]
    original.append(cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),1))

In [5]:
"""This for loop is the base for the algorithm. 
It steps through each frame from the video and 
performs a set of actions on it. At each step, 
we will add an extra step of processing and change 
the cv2.imshow() function to output the frames 
from the most recent step"""
for j, frame in enumerate(original):
    
    # Show each frame in window
    cv2.imshow('Original video',frame)
    
    # Pause to allow viewing of each frame and 
    # allow window to be closed with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# Closes video window when finished 
cv2.destroyWindow('Original video')

Once we have the feed, the next step is figuring out when a train is passing and its direction. Let's break this process down into three questions:

1. Is something moving? 
2. Is that moving thing a train? 
3. In what direction is it moving? 

### Part One: Is something moving? 

The first major step to our algorithm is determing if there is movement in a given frame of the video feed. We do this by developing a model of what we think the stationary background in the video is, and then comparing each new frame to that model to identify areas of change. This process, referred to as background subtraction, consists of pre-processing each frame, comparing the frame to the current model of the background, identifying areas of change, and updating the background model with the current frame.

There are a number of methods that can be used for background subtraction. In this POC, we use the simple, computationally efficient method of frame differencing. However, we will see in future posts that other methods  can also be easily implemented in OpenCV that have different advantages and disadvantages.  

#### 1. Convert each frame to gray scale

Typically, we like to see images and video in color, which requires three pieces of information for every pixel (such as degree of red, blue, and green for RGB images).  However, one channel (i.e. gray-scale) is typically all that is needed for effective background subtraction. Thus, upon reading each frame from the video feed, our first step is to convert it to gray-scale.

Numerous methods exist for converting images to one channel, including simply choosing one of the three color channels, using weighted combinations of the blue, green, and red channels, and using the first principal component from a [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA). For this work, we will use OpenCV's gray-scale function, which uses a weighted combination of the three channels. As each frame, `frame`, of the original video is read, it is converted to gray-scale as follows:

<center><code>
`gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)`
</code></center>

The video feed after this first phase of processing can be viewed in the following video:

In [8]:
"""This is the same for loop as seen previously for reading
and displaying the original video. However, one line has been
added that takes each frame as it is read and converts it to 
gray scale. The cv2.imshow() function has been adjusted to 
output the newly processed gray-scale frame. Running this 
cell would result in the playing of the same video as seen 
directly below."""
for j,frame in enumerate(original):
    
    # Convert frame to gray_scale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('Gray scale',gray_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Gray scale')

#### 2. Smooth each frame

<img src="figures/2dgaussian.png" alt="2D Gaussian" width="150" align="right">
Our goal is to detect differences between frames (i.e. motion) but we do not want to detect any differences due to noise or small scale changes (as we are trying to detect the motion of a large train, not tiny leaves!). Therefore, to reduce noise in each frame, we apply a [Gaussian blur](https://en.wikipedia.org/wiki/Gaussian_blur), which replaces each pixel's value with a weighted average of it and the surrounding pixels. The weights, assigned to each pixel (*x*, *y*), are determined by the two-dimensional Gaussian, pictured to the right and given by the following equation: 

\begin{equation}
G(x,y) = Ae^{\frac{-(x-x_0)^2}{2\sigma_x^{2}}+\frac{-(y-y_0)^2}{2\sigma_y^{2}}}
\end{equation}

where $(x_0, y_0)$ is the location of the pixel in question, $\sigma_x$ and $\sigma_y$ are the standard deviations of the distribution in the *x* and *y* directions, and *A* is a scale parameter chosen such that $\sum_{i} G_i = 1$.
The size of the neighborhood that contributes significantly to a given pixel's new value is determined by the standard deviations, $\sigma_x$ and $\sigma_y$. Theoretically, we could use every pixel in the computing of each single pixel value but practically, very low weight is given to pixels more than ~$3\sigma$ away from a given pixel. In OpenCV's `GaussianBlur` function, we define the size of the neighborhood in the *x* and *y* direction that we want to include in the calculation of each pixel. This size provides control over the computational effort required for the blur. We can then define the $\sigma_x$ and $\sigma_y$ ourselves or simply input `0`, which results in OpenCV calculating $\sigma$ based on the kernel size and a heuristic found to be effective in the literature. In this POC, we allow OpenCV to calculate $\sigma$ using this heuristic and specify a symmetrical kernel, with $k_x = k_y = k$ so that the size of the neighborhood is equal in both the *x* and *y* direction: 
<center>
<code>smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)</code>
</center>

Here are examples of smooth images for various kernel sizes, *k*:

<img src="figures/smooth_images_k.jpg" alt="Smooth images" width="960">

In practice, `k` will depend on the resolution of the image, the quality of the camera, and the scale of what is trying to be detected. It will typically take iterating upon to determine the best value for a given task. In this POC, we iterated to using a `k` of 31 pixels, which will be used in the following steps of the algorithm.

The Gaussian blur is applied to each frame directly after it is converted to gray-scale, and the output of this step is shown in the following video: 

In [14]:
"""This code builds on the last, now smoothing each 
gray-scale frame using a Gaussian blur with kernel size
of k in both the x and y directions. The cv2.imshow()
function now outputs the smoothed image at each loop. If 
executed, the same video as shown directly below would appear."""

k = 31 # Define Gaussian kernel size

for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.
                              COLOR_BGR2GRAY)
    
    # Apply a Gaussian blur to the gray scale frame 
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    cv2.imshow('Smooth', smooth_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
cv2.destroyWindow('Smooth')

#### 3. Create model of background

While more complex methods can require fitted statistical models for each pixel, frame differencing simply requires a representative image as the background model. Basic options for this representative model include a stationary image deemed as background by the user and the average of the last *n* images. In our POC, we employ a running average, which is still simple to implement and requires minimal computation but also is able to adapt to changing conditions over time. The running average is calculated for a given pixel (*x*,*y*) at each time, *t* as:
\begin{equation}
R(x,y,t) = (1-\alpha)R(x,y,t-1) + \alpha F(x,y,t)
\end{equation}
The value $\alpha$ adjusts the weight of the current frame being averaged in and regulates the speed at which the previous frames are forgotten.

To update the running average in Python, we use the function: 
<center>
<code>
cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
</code>
</center>


Since we are summing fractions of images, we need to be able to hold float values for the pixels, which requires us to convert the smooth frame that we are updating the running average with to float values. 


In [11]:
"""Now at each step, we update the running average, 
which we use as the model for the background. If we
are on the first iteration through the for loop, there
is no background model to update and we make it the 
same as the initial smoothed frame."""

alpha = 0.2 # Define weighting coefficient
running_avg = None # Define variable for running average

k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    # Otherwise, update running average with new smooth frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    else:
        cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    cv2.imshow('Running average', cv2.convertScaleAbs(running_avg))

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Running average')

#### 4. Calculate the difference




<center><img src="figures/run-smooth-no-diff73.jpg" alt="Background and foreground" width="860"></center>
<center><img src="figures/diff-bar73.jpg" alt="Background and foreground" width="660"></center>


To detect areas of change between the current frame and the current model of the background, we take the absolute difference between the pixel values of the two frames (i.e. `|smooth_frame - running_avg|`) using the [`cv2.absdiff()`](http://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#absdiff) function:

<center><code>diff = cv2.absdiff(np.float32(smooth_frame), running_avg)</code></center>
<br />

Again, we must covert `smooth_frame` to float values to perform mathematical operations with the pixel values. This step is actually performed *before* updating the background model as we are trying to compare the new frame to our already existing model of the background. Performing this for each incoming frame, we get as output the following video:


In [10]:
"""Here we calculate the difference in pixel values between 
the current frame and the background model. Note, we calculate this difference *before* 
we update the background model."""

alpha = 0.2 
running_avg = None 
k = 31
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
        
    # Find |difference| between current smoothed frame and running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)

    cv2.imshow('Difference', diff)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Difference')

#### 5. Threshold the difference

As we see in the above video, we detect variations in pixel values in locations and at times other than where and when the train is passing, such as when leaves are rustling. One way to remove some of the small-scale movements is to count only large fluctuations in pixel values as motion. We do this by setting a threshold value and setting all differences higher than this value as motion using the [`cv2.threshold()`](http://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html?highlight=threshold#threshold) function:
<center><code>_, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)</code></center>
<br />



This function sets any value in `diff` that is greater than `motion_thresh` to 1 and leaves all other pixels as zero. Applying this function to the prior video of differences using a threshold of 35, we get the following video, where white area denotes motion (`diff > 35`) and black area is considered stationary (`diff <=35`):

In [12]:
"""Here, for each iteration of the for loop, we 
take the diff frame and change any pixels greater
than motion_thresh to a value of 1 and any pixels
less than or equal motion_thresh to a value of 0."""

# All pixels with differences above this value will be set to 1
motion_thresh = 35 

alpha = 0.2
running_avg = None 
k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 1, otherwise 0
    _, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)
    
    cv2.imshow('Thresholded difference', subtracted)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Thresholded difference')

You will note in the above video that after the train first enters the frame, the only motion that ends up being detected is where the windows are located. This behavior is due to the fact that the train outside of the row of windows does not greatly vary in color. In other motion detection applications, such as detecting the passing of people or cars, the objects do not take up the entire frame and typically do not continue to move across the frame for extended periods of time, so their detection will be a lot more defined spatially. If you are trying this code at home, you can use the code in the appendix to test this out using your computer's camera and your own motion. 

### Part two: Is that moving thing a train? 

Now that we have identified where there is motion in the frame, we now need to figure out if that motion is in fact a train. Two questions that can help us figure out if the motion is in fact a train and not a person or car are: 
1. Is enough of the frame in motion? 
2. Does this motion last as long as a train? 

To consider the first question, we can look at the fraction of each frame that is in motion, $\frac{\textrm{number of pixels in motion}}{\textrm{total number of pixels}}$ as shown in the following video alongside the thresholded video: 

In [ ]:
"""Here we compute the fraction of each frame 
that is identified to be in motion. The updated
plot of fraction of motion versus frame number is 
output alongside the video of the thresholded 
image below."""

# Initialize list for capturing fraction of each frame in motion 
motion_fractions = [] 

motion_thresh = 35 
alpha = 0.2
running_avg = None 
k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (g,g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)
    
    # Calculate the fraction of pixels where motion is detected
    # i.e. where subtracted==1
    motion_fraction = (sum(sum(subtracted))/
                       (subtracted.shape[0]*subtracted.shape[1]))

    motion_fractions.append(motion_fraction)
    


We can see that despite only the windows being in motion for a majority of the train's length, we still see a clear portion of the frame in motion. By setting a threshold for how much of the frame must be in motion, we can say "whatever is in the frame is *large* enough to be a train," such as by the horizontal line in the following figure: 

<img src="figures/green-frame-fraction-with-horizontal.jpg" alt="Area threshold" width="860">

Now that we have identified there is enough motion to be a train, we must determine if the motion lasts long enough to be considered a train so that we do not count any pedestrians or cars passing by. We see in the following figure that at the current frame rate (e.g. number of images per second), a train lasts for approximately 70 frames. 

<img src="figures/green-frame-fraction-with-horizontal-and-vertical.jpg" alt="Area threshold" width="860">

In [ ]:
"""Here, we test whether the current frame has 
enough motion detected to be considered big enough
to be a train and we look at the previous frames and 
test whether there have been enough positive motion 
frames to be considered long enough to be a train."""

# Initialize dataframe to capture motion
history = pd.DataFrame(data=[[0,0]],columns=['Fraction','In_motion'])
# Define Minimum number of frames to be considered a train
history_length = 70
# Define minimum fraction of frame to be in motion to be 
# considered train style 
frame_thresh = 0.025

motion_thresh = 35 
alpha = 0.2
running_avg = None 
k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (g,g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    _, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)
    
    motion_fraction = (sum(sum(subtracted))/
                       (subtracted.shape[0]*subtracted.shape[1]))
    
    history.loc[len(history)+1] = [motion_fraction, motion_fraction>frame_thresh]
    
    detect = (history.tail(history_length).In_motion.sum()==history_length)
    
    if detect: 
        print 'Train detected beginning at frame', j-history_length
        # Reset history
        history = pd.DataFrame(data=[[0,0]],columns=['Fraction','In_motion']) 

Thus, by applying required thresholds in fraction of frame in motion and number of frames where that threshold is met, we can detect that a train passes. 

### Part three: In what direction is it moving? 

However, this method does not tell us in what direction the train is moving. For that piece of the problem, we have to extend the algorithm. We see in the image below that when the train first enters the frame, it is only present on one side. If the train is south-bound (such as in the figure), the train will first only be present in the left half of the frame versus the right side if north-bound. 

<img src="figures/original73.jpg" alt="Area threshold" width="560">

To detect whether the train enters from the left or right side, we can identify motion in two regions of interest (ROIs) rather than in the entire frame: 

<img src="figures/roiframe073.jpg" alt="Area threshold" width="560">

Now, plotting the fraction of each ROI over time, we see in the following video that motion sets off in the left ROI two frames before motion in right ROI:

There are a number of ways we can use these two streams of data (fraction of motion in left ROI and fraction of motion in right ROI) to determine that a train is passing and in what direction it is going. In this POC, we simply track whether each ROI has detected the required fraction of motion in a dataframe (see below) and when one ROI has met this threshold for enough frames to be considered a train, we identify which ROI has detected motion longer. Within the algorithm, the steps are as follows: 

1. Initialize a dataframe, `history`.
2. At each frame, *t*, add row to dataframe indicating if left and/or right meets motion threshold.
3. Repeat #2 until motion threshold met on either side for last *T* frames.
3. The side with the detection for all *T* indicates the train's direction.
4. Reinitialize the `history` dataframe to detect future trains.
5. Return to #2.     
<img src="figures/left-and-right.jpg" alt="Area threshold" width="420" align='right'>
<img src="figures/train-history.png" alt="Area threshold" width="200" align='left'>

This is a simple way for doing this direction detection to prove out the general idea. In later posts, we will show more robust methods that can be used for real-time deployment. 

In [ ]:
"""Here, we now track the fraction of each ROI is in motion,
determine if enough frames in a row have had at least one ROI 
meet the motion threshold and then figure out which ROI
has been in motion longer to determine the train'direction."""

# Define Minimum fraction of ROI to be considered in motion
roi_thresh = 0.05 

history = pd.DataFrame(columns=['Detected left','Detected right'])
history_length = 70
running_avg = None 
motion_thresh = 35 
alpha = 0.2
k = 31
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)

    left_area = subtracted[270:400,240:340] 
    right_area = subtracted[270:400,540:640] 
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    # Update the history with whether the train detection criteria was met on either side
    history.loc[len(history)+1] = [left_fraction>roi_thresh, right_fraction>roi_thresh]
    
    # Get how many of last n frames had a train detected for left and right ROIs
    left_cum, right_cum = history.tail(history_length).sum()
    
    # If all of last n frames had train detected on at least one side
    if left_cum >= history_length or right_cum >= history_length: 
        
        # If a train was detected longer on the left, then it is south bound
        if left_cum > right_cum:
            print 'South bound train beginning at frame', j-history_length
        else: 
            print 'North bound train beginning at frame', j-history_length
        
        train = history.tail(history_length).head(10)
        
        # Reset the history to be able to detect a new train
        history = pd.DataFrame(columns=['Detected left','Detected right'])


## Bringing it all together

The end product is a model for detecting the train given a set of frames. In practice, we must train the model on labeled training data to estimate values for the following parameters:
* `k`: gives the size of the kernel used to perform the Gaussian blur for smoothing the image. Will change according to the resolution and quality of the image and scale of the motion being detected.
* `alpha`: adjusts how much the previous frames are weighted in the `running_avg`. Will change according to frame rate of camera and how quickly the background may change.
* `motion_thresh`: is the cut-off for the value of the difference between a pixel in the current frame and moving average to be considered moving. 
* `roi_thresh`: the fraction of the ROI that must be considered in motion to imply a train's passing.
* `history_length`: Number of frames train has to be detected in for it to be considered a Caltrain. Depends on frame rate and length of train to be detected. 

## Conclusions

So our proof of concept is complete — we can detect a train's passing and direction from video! Now, we just need to iterate on this algorithm so that it can be: 
1. Run in real-time
2. Deployed on a Raspberry Pi, which has limited memory and computational power
3. Adaptable to changes in lighting

Look out for the future blog posts which address these requirements. 

## Appendix: Trying out motion detection at home

You can try out the motion detection algorithms shown here using your own web cam! Use the following code to start. Simply press 'q' on your keyboard to end the demo.  

```python
# Start streaming images from your computer camera
feed = cv2.VideoCapture(0) 

# Define the parameters needed for motion detection
alpha = 0.02 # Define weighting coefficient for running average
motion_thresh = 35 # Threshold for what difference in pixels  
running_avg = None # Initialize variable for running average

k = 31
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    # Find absolute difference between the current smoothed frame and the running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 255, otherwise 0
    _, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)
    
    cv2.imshow('Actual image', current_frame)
    cv2.imshow('Gray-scale', gray_frame)
    cv2.imshow('Smooth', smooth_frame)
    cv2.imshow('Difference', diff)
    cv2.imshow('Thresholded difference', subtracted)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cv2.destroyAllWindows()
feed.release()
```

In [5]:
# Start streaming images from your computer camera
feed = cv2.VideoCapture(0) 

# Define the parameters needed for motion detection
alpha = 0.02 # Define weighting coefficient for running average
motion_thresh = 35 # Threshold for what difference in pixels  
running_avg = None # Initialize variable for running average

k = 31
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    # Find absolute difference between the current smoothed frame and the running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 255, otherwise 0
    _, subtracted = cv2.threshold(diff, motion_thresh, 1, cv2.THRESH_BINARY)
    
    cv2.imshow('Actual image', current_frame)
    cv2.imshow('Gray-scale', gray_frame)
    cv2.imshow('Smooth', smooth_frame)
    cv2.imshow('Difference', diff)
    cv2.imshow('Thresholded difference', subtracted)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cv2.destroyAllWindows()
feed.release()